In [2]:
import  requests

In [19]:
url = "http://fas/api/v1/get-api/request3?name=encore&email=encore@encore.com&organization=public"

In [20]:
r = requests.get(url)

In [21]:
r.status_code

200

In [22]:
r.text

"{'name' :'encore', 'email' :'encore@encore.com', 'organization' :'public'}"

In [24]:
r.json()

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [29]:
from urllib import parse

data = parse.urlparse(url)
data

ParseResult(scheme='http', netloc='fas', path='/api/v1/get-api/request3', params='', query='name=encore&email=encore@encore.com&organization=public', fragment='')

In [28]:
data.query

'name=encore&email=encore@encore.com&organization=public'

In [33]:
query_data = parse.parse_qs(data.query)
query_data

{'name': ['encore'],
 'email': ['encore@encore.com'],
 'organization': ['public']}

In [35]:
query_data['name'] = 'bts'
query_data

{'name': 'bts', 'email': ['encore@encore.com'], 'organization': ['public']}

In [40]:
parse.urlencode(query_data, doseq=True) # doseq -> 딕셔너리 값의 리스트 형태를 유지할지를 묻는 것
                                        # True : 리스트 형태 X, 무시 / False : 리스트 형태 유지

'name=bts&email=encore%40encore.com&organization=public'

In [49]:
# POST 방식
post_url = "http://fas/api/v1/post-api/domain"

requests.post(post_url)

<Response [200]>

In [53]:
# 공공 데이터 포털에서 REST API로 제공된 대기오염 관련 XML 형식으로 받은 데이터 활용
## https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15073861
### 위 링크에서 가져온 데이터.
import requests

url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&returnType=xml&numOfRows=10000&pageNo=1&stationName=%EC%A2%85%EB%A1%9C%EA%B5%AC&dataTerm=3MONTH&ver=1.0"
# 종로구의 대기 오염 상태

r3 = requests.get(url)


In [58]:
r3.text[1000:2000]

'm10Flag/>\n        <pm10Value>24</pm10Value>\n        <o3Grade>2</o3Grade>\n        <khaiGrade>2</khaiGrade>\n        <pm25Value>16</pm25Value>\n        <no2Flag/>\n        <no2Grade>1</no2Grade>\n        <o3Flag/>\n        <pm25Grade>1</pm25Grade>\n        <so2Flag/>\n        <dataTime>2023-08-17 13:00</dataTime>\n        <coGrade>1</coGrade>\n        <no2Value>0.013</no2Value>\n        <pm10Grade>1</pm10Grade>\n        <o3Value>0.076</o3Value>\n      </item>\n      <item>\n        <so2Grade>1</so2Grade>\n        <coFlag/>\n        <khaiValue>79</khaiValue>\n        <so2Value>0.003</so2Value>\n        <coValue>0.5</coValue>\n        <pm25Flag/>\n        <pm10Flag/>\n        <pm10Value>23</pm10Value>\n        <o3Grade>2</o3Grade>\n        <khaiGrade>2</khaiGrade>\n        <pm25Value>14</pm25Value>\n        <no2Flag/>\n        <no2Grade>1</no2Grade>\n        <o3Flag/>\n        <pm25Grade>1</pm25Grade>\n        <so2Flag/>\n        <dataTime>2023-08-17 12:00</dataTime>\n        <coGrade>

In [59]:
## 위와 같은 결과값 / 선언하는 방식만 다를뿐
import requests

url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'
params ={'serviceKey' : '34lIyWAciAsJlGtIZ4ltpLy2sLZDR+BRWvAv8RgoADNEd+KCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ==', 'returnType' : 'xml', 'numOfRows' : '10000', 'pageNo' : '1', 'stationName' : '종로구', 'dataTerm' : '3MONTH' }

response = requests.get(url, params=params)

In [70]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r3.text)
root

<Element 'response' at 0x000001F41A6C2020>

In [71]:
# pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [73]:
import tqdm # 긴 반복문 같은 오랜 기달림이 필요한 작업에서 진행 시간이 나오게 하는 패키지.
import pandas as pd

In [74]:
items = root.iter(tag = "item")
total = []

for i in tqdm.tqdm(items):
    tmp = {}
    for j in i:
        tmp[j.tag] = j.text
    total.append(tmp)
    
df = pd.DataFrame(total)

0it [00:00, ?it/s]

2207it [00:00, 184549.71it/s]


In [78]:
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15073861
##  측정소별 실시간 측정정보 조회 목록에서 아래 컬럼들의 의미 파악 가능.
pd.set_option('display.max_columns', None)

df


,so2Grade,coFlag,khaiValue,so2Value,coValue,pm25Flag,pm10Flag,pm10Value,o3Grade,khaiGrade,pm25Value,no2Flag,no2Grade,o3Flag,pm25Grade,so2Flag,dataTime,coGrade,no2Value,pm10Grade,o3Value
0,1,None,96,0.003,0.5,None,None,22,2,2,15,None,1,None,1,None,2023-08-17 14:00,1,0.013,1,0.085
1,1,None,88,0.003,0.5,None,None,24,2,2,16,None,1,None,1,None,2023-08-17 13:00,1,0.013,1,0.076
2,1,None,79,0.003,0.5,None,None,23,2,2,14,None,1,None,1,None,2023-08-17 12:00,1,0.016,1,0.064
3,1,None,65,0.003,0.5,None,None,18,2,2,9,None,1,None,1,None,2023-08-17 11:00,1,0.013,1,0.048
4,1,None,54,0.003,0.4,None,None,18,2,2,13,None,1,None,1,None,2023-08-17 10:00,1,0.014,1,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,1,None,58,0.003,0.4,None,None,34,2,2,13,None,1,None,1,None,2023-05-17 20:00,1,0.012,2,0.033
2203,1,None,59,0.003,0.3,None,None,34,2,2,9,None,1,None,1,None,2023-05-17 19:00,1,0.012,2,0.037
2204,1,None,60,0.003,0.3,None,None,41,2,2,12,None,1,None,1,None,2023-05-17 18:00,1,0.011,2,0.040
2205,1,None,72,0.003,0.3,None,None,48,2,2,13,None,1,None,1,None,2023-05-17 17:00,1,0.009,2,0.055


In [ ]:
##  종로구, 동작구, 강남구 각 데이터 총합

import requests
import tqdm # 긴 반복문 같은 오랜 기달림이 필요한 작업에서 진행 시간이 나오게 하는 패키지.
import pandas as pd
import xml.etree.ElementTree as ET


In [79]:
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty'

params1 ={'serviceKey' : '34lIyWAciAsJlGtIZ4ltpLy2sLZDR+BRWvAv8RgoADNEd+KCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ==', 'returnType' : 'xml', 'numOfRows' : '10000', 'pageNo' : '1', 
         'stationName' : '종로구', 'dataTerm' : '3MONTH' }

params2 ={'serviceKey' : '34lIyWAciAsJlGtIZ4ltpLy2sLZDR+BRWvAv8RgoADNEd+KCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ==', 'returnType' : 'xml', 'numOfRows' : '10000', 'pageNo' : '1', 
         'stationName' : '동작구', 'dataTerm' : '3MONTH' }

params3 ={'serviceKey' : '34lIyWAciAsJlGtIZ4ltpLy2sLZDR+BRWvAv8RgoADNEd+KCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ==', 'returnType' : 'xml', 'numOfRows' : '10000', 'pageNo' : '1', 
         'stationName' : '강남구', 'dataTerm' : '3MONTH' }

air_r1 = requests.get(url, params=params1)
air_r2 = requests.get(url, params=params2)
air_r3 = requests.get(url, params=params3)

In [80]:
root1 = ET.fromstring(air_r1.text)
root2 = ET.fromstring(air_r2.text)
root3 = ET.fromstring(air_r3.text)

In [87]:
items1 = root1.iter(tag = "item")
items2 = root2.iter(tag = "item")
items3 = root3.iter(tag = "item")

t_items = [items1, items2, items3]

total1 = []
total2 = []
total3 = []

# t_total = [total1, total2, total3]

# for x in tqdm.tqdm(t_items):
#     for i in x:
#         tmp = {}
#         for j in x[i]:
#             tmp[j.tag] = j.text
#         t_total[i].append(tmp)
    
for i in items1:
    tmp = {}
    for j in i:
        tmp[j.tag] = j.text
    total1.append(tmp)

for i in items2:
    tmp = {}
    for j in i:
        tmp[j.tag] = j.text
    total2.append(tmp)

for i in items3:
    tmp = {}
    for j in i:
        tmp[j.tag] = j.text
    total3.append(tmp)
    
jongro_df = pd.DataFrame(total1)
dongjak_df = pd.DataFrame(total2)
gangnam_df = pd.DataFrame(total3)

In [107]:
jongro_df['지역'] = '종로구'
jongro_df.shape

(2206, 19)

In [108]:
dongjak_df['지역'] = '동작구'
dongjak_df.shape

(2206, 19)

In [110]:
gangnam_df['지역'] = '강남구'
gangnam_df.shape

(2206, 19)

In [96]:
total_df = []
total_df.append(jongro_df)
total_df.append(dongjak_df)
total_df.append(gangnam_df)

In [105]:
air_df2 = pd.concat(total_df)
air_df2.shape

(6618, 19)

##  종로구, 동작구, 강남구 각 데이터 총합 : 정답 코드 아래!

In [100]:
## 위와 같은 수행을 위한 코드
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm


import requests

url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?serviceKey=34lIyWAciAsJlGtIZ4ltpLy2sLZDR%2BBRWvAv8RgoADNEd%2BKCgHe84XiSRUwL8JMMIubzsFW3ddjcNlhZHhvJIQ%3D%3D&returnType=xml&numOfRows=10000&pageNo=1&stationName={}&dataTerm=3MONTH&ver=1.0"

station = ['종로구', '강남구', '동작구']
df_total = []
for city in tqdm.tqdm(station):
    r3 = requests.get(url.format(city), verify=False)
    root = ET.fromstring(r3.text)

    items = root.iter(tag='item')
    total = []
    for data in items:
        tmp = {}
        for x in data:
            tmp[x.tag] = x.text
        total.append(tmp)

    df = pd.DataFrame(total)
    df['지역'] = city
    df_total.append(df)
air_df = pd.concat(df_total)

100%|██████████| 3/3 [00:29<00:00,  9.76s/it]


In [113]:
air_df.dropna(thresh=3000, axis=1, inplace=True)

In [114]:
air_df.shape

(6621, 16)

In [136]:
air_df.loc[air_df.o3Value != "-", 'o3Value'][0].values[0]
##  String으로 되어있음.

'0.091'

In [137]:
tmp_df = air_df.loc[(air_df.dataTime.apply(lambda x : x[:7]  == "2023-08")) & (air_df.o3Value != "-") ,:].copy()


In [139]:
tmp_df['o3Value'] = tmp_df.o3Value.astype(float)
# '-' 가 포함한 행들은 전부 제외한 상태에서 float 형태로 바꿈 

In [140]:
tmp_df.groupby(['지역'])[['o3Value']].agg(['mean', 'max', 'min', 'median'])

o3Value                     
         mean    max    min median
지역                                
강남구  0.031981  0.131  0.002  0.025
동작구  0.037126  0.141  0.005  0.030
종로구  0.034927  0.126  0.005  0.028

In [141]:
# db 저장
import sqlalchemy
# DB 설정
user = 'root'
password = '8489'
host = '127.0.0.1' # loopback IP or Localhost 
port = '3306'

## MySQL 워크벤치에서 직접 'encore' 데이터베이스를 생성
database = 'encore'
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

## mysql에 연결되고 starbucks 테이블이 생성되고 데이터가 저장됨
air_df.to_sql(con=engine, name="air", if_exists="replace")

6621